# Universidad del Valle de Guatemala
## Security Data Science - 10
* Jose Abraham Gutierrez Corado - 19111
* Javier Alejandro Cotto Argueta - 19324

## Laboratorio 1: Detección de Pishing

### Importación de Librearias

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
#from yellowbrick.features import Rank2D
#from yellowbrick.features import RadViz

### Explorar Datos

In [2]:
df = pd.read_csv('dataset_pishing.csv')
df.head(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,0,1,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,1,0,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,1,0,1,0,0,0,0,0,1,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,0,0,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,0,1,0,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [3]:
df[df.status == 'legitimate'].sample(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
2038,http://rgzbgroup.hk,0,0,0,0,0,0,0,0,0,...,1,1,0,65,-1,0,0,0,0,legitimate
4267,http://www.coolmath-games.com/0-help-me,0,1,0,0,0,0,0,0,0,...,1,1,0,1030,5908,14374,0,0,5,legitimate
6084,https://www.decisivetactics.com/support/view?a...,0,1,0,0,0,0,0,0,0,...,1,1,0,1780,4428,1059659,0,0,4,legitimate
9991,http://www.horniman.ac.uk/dinosaurs,0,1,0,0,0,0,0,0,0,...,0,0,0,0,-1,2639634,0,1,5,legitimate
8044,http://www.consumersearch.com/color-laser-prin...,0,1,0,0,0,0,0,0,0,...,1,1,0,666,8571,22192,0,0,5,legitimate


In [4]:
df[df.status == 'phishing'].sample(5)

,url,ip,nb_www,nb_com,nb_dslash,http_in_path,punycode,port,tld_in_path,tld_in_subdomain,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
605,https://diabloiptvreseller.com/payment/,0,0,0,0,0,0,0,0,0,...,1,0,0,60,-1,0,0,1,0,phishing
2409,http://quadfabrik.de/login.stratoserver.net.cp...,1,0,0,0,0,0,0,0,0,...,1,0,0,0,-1,0,0,1,0,phishing
1949,http://172.217.21.162/pixel/,1,0,0,0,0,0,0,1,1,...,1,0,1,0,-1,0,1,1,0,phishing
853,http://www.allflagsauto.com/Contents/,0,1,0,0,0,0,0,0,0,...,0,1,0,47,1048,0,0,1,0,phishing
7174,http://accountchckecker-update-now.drpiza.com/...,0,0,0,0,0,0,0,0,0,...,1,0,0,860,3157,0,0,1,1,phishing


Si se encuentra balanceado.

In [5]:
df.shape

(11430, 67)

### Derivación de Características
Las características del articulo son:
* f1: Full URL length
* f2: Hostname length
* f4-f20: Special Characters in URL
* f25: HTTPS Token
* f26: Ratio of digits in full URLs
* f27: Ratio of digits in hostnames

In [6]:
# Creamos un nuevo dataframe para que sea más fácil la manipulación de datos
dfNew = df[['url', 'status']]
dfNew.head(5)

,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


### Listado de Partes de una URL
* Protocolo HTTPS
* Subdominio
* Dominio
* TLD
* Ruta
* Parámetro
* Etiqueta

### Funciones

In [7]:
# Obtenemos F1
dfNew['length'] = dfNew['url'].str.len()

C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_15896\3796037450.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['length'] = dfNew['url'].str.len()


In [8]:
#Obtenemos F2
from urllib.parse import urlparse

def getHostname(dfNew):
    hostnames = []
    for url in dfNew.url:
        hostnames.append('{uri.scheme}://{uri.netloc}/'.format(uri=urlparse(url)))
    
    dfNew['hostname_length'] = hostnames
    
    dfNew['hostname_length'] = dfNew['hostname_length'].str.len()
    
    return dfNew

#Obtenemos la columna de las carateristicas F4 - F20
def SpecialCharacters(dfNew):
    sChars = [".","-", "@", "?", "&", "|", "=", "_", "~", "%", "/", "*", ":", ",", ";", "$", "%20", " "]
    cont = 0
    scCount = []
    for url in dfNew.url:
        for c in url:
            if (c in sChars) and (c != "%"):
                cont += 1
                
            elif c == "%":
                c2 = url.index(c)
                c2 = c + url[c2]
                
                if c2 == "%2":
                    cont += 1
                else:
                    cont += 1
                    
        scCount.append(cont)
        cont = 0
        
    dfNew['Special_Char_Count'] = scCount
    
    return dfNew

#Obtenemos si la URL tiene HTTPS o no
def Check4HTTPS(dfNew):
    check = []
    for url in dfNew.url:
        if "https" in url:
            check.append(1)
        else:
            check.append(0)
    
    dfNew['HTTPS'] = check
    return dfNew

#Obtenemos el ratio de números en la URL y en el Hostname
def ratioDigits(dfNew):
    digits = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    hostnames = []
    urlDigits = []
    hostnameDigits = []
    cont1 = 0
    cont2 = 0
    
    for url in dfNew.url:
        for c in url:
            if c in digits:
                cont1 += 1
                
        urlDigits.append(cont1/len(url))
        cont1 = 0
        hostnames.append('{uri.scheme}://{uri.netloc}/'.format(uri=urlparse(url)))
    
    for hn in hostnames:
        for c in hn:
            if c in digits:
                cont2 += 1
                
        hostnameDigits.append(cont2/len(hn))
        cont2 = 0
        
    dfNew['urlDigits_ratio'] = urlDigits
    dfNew['hostnameDigits_ratio'] = hostnameDigits
    
    return dfNew

### Ejecutar las funciones en el dataset

In [9]:
dfNew = getHostname(dfNew)
dfNew = SpecialCharacters(dfNew)
dfNew = Check4HTTPS(dfNew)
dfNew = ratioDigits(dfNew)

C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_15896\1161462486.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['hostname_length'] = hostnames
C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_15896\1161462486.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['hostname_length'] = dfNew['hostname_length'].str.len()
C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_15896\1161462486.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

Dataframe resultante con las nuevas columnas

In [10]:
dfNew.sample(5)

,url,status,length,hostname_length,Special_Char_Count,HTTPS,urlDigits_ratio,hostnameDigits_ratio
10804,https://en.wikipedia.org/wiki/John_Peel_(hunts...,legitimate,50,25,9,1,0.0,0.0
8981,https://www.chhotumaharaj.com/.well-known/pki-...,phishing,94,30,16,1,0.0,0.0
1190,http://www.emedicinehealth.com/neuropathy/arti...,legitimate,56,31,9,0,0.0,0.0
10102,http://www.gothic.net/,legitimate,22,22,6,0,0.0,0.0
8383,https://ontslag.nl/,legitimate,19,19,5,1,0.0,0.0


### Preprocesamiento
Eliminación de columna Dominio y status de categorica a binaria

In [11]:
dfNew['status'] = df['status'].replace(['legitimate', 'phishing'], ['0', '1'])
dfNew.drop(['url'], axis=1, inplace=True)

C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_15896\928132545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['status'] = df['status'].replace(['legitimate', 'phishing'], ['0', '1'])
C:\Users\Javier Cotto\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
dfNew.sample(5)

,status,length,hostname_length,Special_Char_Count,HTTPS,urlDigits_ratio,hostnameDigits_ratio
7506,1,64,22,12,0,0.031250,0.000000
7510,0,77,28,15,0,0.012987,0.000000
1196,1,32,33,5,0,0.062500,0.060606
1079,0,27,19,5,0,0.000000,0.000000
9558,1,15,12,5,0,0.133333,0.083333


### Visualización de Resultados

In [16]:
from ydata_profiling import ProfileReport

profile = ProfileReport(dfNew, title="Detecting Pishing in URLs")
profile.to_file("Pishing.html")

C:\Users\Javier Cotto\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
Export report to file: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 125.34it/s]
